In [ ]:
# download API key from google drive
## Original: https://colab.research.google.com/drive/1eufc8aNCdjHbrBhuy7M7X6BGyzAyRbrF#scrollTo=y5_288BYp6H1
## When you run for the first time, you will see a link to authenticate.

# from googleapiclient.discovery import build
# import io, os
# from googleapiclient.http import MediaIoBaseDownload
# from google.colab import auth

# auth.authenticate_user()

# drive_service = build('drive', 'v3')
# results = drive_service.files().list(
#         q="name = 'kaggle.json'", fields="files(id)").execute()
# kaggle_api_key = results.get('files', [])

# filename = "/root/.kaggle/kaggle.json"
# os.makedirs(os.path.dirname(filename), exist_ok=True)

# request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
# fh = io.FileIO(filename, 'wb')
# downloader = MediaIoBaseDownload(fh, request)
# done = False
# while done is False:
#     status, done = downloader.next_chunk()
#     print("Download %d%%." % int(status.progress() * 100))
# os.chmod(filename, 600)

In [ ]:
# 開催中のcompetitionのlistを表示
# !kaggle competitions list

In [ ]:
# competition のデータをダウンロード
# !kaggle competitions download -c us-patent-phrase-to-phrase-matching

In [ ]:
# import zipfile

# titanic_zip = "/content/us-patent-phrase-to-phrase-matching.zip"

# # zipファイルに含まれているファイルの取り出す
# with zipfile.ZipFile(titanic_zip) as myzip:
#     myzip.extractall()    # zipファイルに含まれているファイルを全て取り出す

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '') #環境変数を取得

# #getting my own machine
# creds = '{"username":"toya18","key":"c91a95a0f3c7272a0ba5b4d3b5406f74"}' 

In [ ]:
# for working with paths in Python, I recommend using `pathlib.Path`
# from pathlib import Path

# cred_path = Path('~/.kaggle/kaggle.json').expanduser()
# if not cred_path.exists():
#     cred_path.parent.mkdir(exist_ok=True)
#     cred_path.write_text(creds)
#     cred_path.chmod(0o600)

In [ ]:
# path = Path('us-patent-phrase-to-phrase-matching')

# if not iskaggle and not path.exists():
#     import zipfile,kaggle
#     kaggle.api.competition_download_cli(str(path))
#     zipfile.ZipFile(f'{path}.zip').extractall(path)

In [ ]:
# if iskaggle:
#     path = Path('../input/us-patent-phrase-to-phrase-matching')
#     ! pip install -q datasets
    
# !ls {path}

In [ ]:
# !pip install --upgrade pip
# !pip install git+https://github.com/huggingface/transformers
# !pip list | grep -E 'transformers|tokenizers'

In [ ]:
!pip install transformers
!pip install datasets

In [ ]:
# インターネットでダウンロードできる時に実行
# from huggingface_hub import snapshot_download
# download_path = snapshot_download(cache_dir="./your/path/bigscience_t0", repo_id="rinna/japanese-gpt2-xsmall") #事前学習モデルをダウンロード

In [ ]:
# from transformers import AutoConfig
# from huggingface_hub import hf_hub_download
from os import environ
# assert environ["TRANSFORMERS_OFFLINE"] == "1"

# hf_hub_download(repo_id="bigscience/T0_3B", filename="config.json", cache_dir="./your/path/bigscience_t0")
# config = AutoConfig.from_pretrained("/content/your/path/bigscience_t0/rinna--japanese-gpt2-xsmall.main.e2dac72065c0da14d687ade9931549711e1f35fd/config.json")

In [ ]:
path = ("../input/us-patent-phrase-to-phrase-matching/")
train = pd.read_csv(path + "train.csv")
test = pd.read_csv(path + "test.csv")
sample_sub = pd.read_csv(path + "sample_submission.csv")

In [ ]:
train.describe(include="object")

In [ ]:
train['input'] = 'TEXT1: ' + train.context + '; TEXT2: ' + train.target + "; ANC1: "+ train.anchor
train["input"][0]

In [ ]:
from datasets import Dataset, DatasetDict
train_ds = Dataset.from_pandas(train)

In [ ]:
from transformers import BertModel, BertConfig

# Initializing a BERT bert-base-uncased style configuration
configuration = BertConfig()

# Initializing a model from the bert-base-uncased style configuration
model = BertModel(configuration)

# Accessing the model configuration
configuration = model.config

In [ ]:
from transformers import BertTokenizer, BertModel
from transformers import AutoModelForSequenceClassification,AutoTokenizer
import torch

model_name = "bert-base-uncased"
#トークン化するモデルを設定
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

DEMO

In [ ]:
# "input"をトークン化してtensor型にする
tokz_demo = tokenizer("I go to school.",return_tensors="pt")

# # model にfit
outputs_demo = model(**tokz_demo)

last_hidden_states = outputs_demo.last_hidden_state
last_hidden_states

In [ ]:
tokz_demo

In [ ]:
def tokz_func(data): return tokenizer(data["input"]) #tokenize する関数を定義

#train_ds をトークナイズしてコーパスをエンコード
tok_ds = train_ds.map(tokz_func, batched=True)

In [ ]:
#tok_ds内を確認
row = tok_ds[0]
print(row)

In [ ]:
#scoreをlabel に置き換え
tok_ds = tok_ds.rename_columns({'score':'labels'})
tok_ds

In [ ]:
# testデータでもtrainデータと同じことをする
test['input'] = 'TEXT1: ' + test.context + '; TEXT2: ' + test.target + '; ANC1: ' + test.anchor
test_ds = Dataset.from_pandas(test)
tok_ds_test = test_ds.map(tokz_func, batched=True)

In [ ]:
def corr(x,y): return np.corrcoef(x,y)[0][1] #相関係数を計算
def corr_d(eval_pred): return {'pearson': corr(*eval_pred)}  #予測した値の相関係数を出力

In [ ]:
# !pip install transformers

In [ ]:
from transformers import TrainingArguments,Trainer

bs = 128
epochs = 4
lr = 1e-6

In [ ]:
 #トレーニング引数を指定
args = TrainingArguments('outputs',
                         learning_rate=lr,
                         warmup_ratio=0.1,
                         lr_scheduler_type='cosine', 
                         fp16=True,  #CPUを使うかGPUを使うかで
                         evaluation_strategy="epoch", 
                         per_device_train_batch_size=bs,
                         per_device_eval_batch_size=bs*2,
                         num_train_epochs=epochs,
                         weight_decay=0.01,  #strength of weight decay
                         report_to='none'
                         )

In [ ]:
# from os import environ
model_name = "bert-base-uncased"
model = BertModel.from_pretrained(model_name, config=configuration) # トークン化するためのモデル
model_nm = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=1) #BERTモデルとその事前トレーニング済みの重みをダウンロード
# トークン化したデータを学習
trainer = Trainer(model_nm,
                  args,
                  train_dataset=tok_ds, 
                  eval_dataset=tok_ds_test,
                  tokenizer=tokenizer,
                  compute_metrics=corr_d)

In [ ]:
trainer.train();

In [ ]:
preds = trainer.predict(tok_ds_test).predictions
print(preds)

In [ ]:
import datasets

submission = pd.DataFrame(columns=["id","score"])
submission["id"] = tok_ds_test['id']
submission["score"] = preds.astype(float)

In [ ]:
submission

In [ ]:
submission.to_csv("submission.csv", index = False) #csvファイル作成